In [5]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [6]:
# reset compute graph

tf.reset_default_graph()

In [7]:
file_name = 'data_for_truncated_rnn.bin'

with open(file_name, 'rb') as f:
    X_train = np.load(f).astype(np.float32)
    y_train_onehot = np.load(f).astype(np.float32)
    X_valid = np.load(f).astype(np.float32)
    y_valid_onehot = np.load(f).astype(np.float32)
    X_test = np.load(f).astype(np.float32)
    y_test_onehot = np.load(f).astype(np.float32)
    
print(X_train.shape)
print(y_train_onehot.shape)
print(X_valid.shape)
print(y_valid_onehot.shape)
print(X_test.shape)
print(y_test_onehot.shape)

(17694, 10, 53)
(17694, 10, 3)
(2212, 10, 53)
(2212, 10, 3)
(2212, 10, 53)
(2212, 10, 3)


In [8]:
## neural network parameters and architecture

# network parameters

batch_size = 1
input_dim = 53
output_dim = 3
num_steps = 10
state_size = 100
num_layers = 3
learning_rate = 1e-4
global_dropout = .8

# feed_dict placeholders

X = tf.placeholder(tf.float32, [batch_size, num_steps, input_dim], name='X')
y = tf.placeholder(tf.float32, [batch_size, num_steps, output_dim], name='y')
keep_prob = tf.placeholder_with_default(1.0, shape=(), name='keep_prob')

# forward pass

input_series = tf.unstack(X, axis=1)
label_series = tf.unstack(y, axis=1)

def rnn_cell_with_dropout():
    return tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicRNNCell(num_units=state_size), 
        input_keep_prob=keep_prob)

cell = tf.nn.rnn_cell.DropoutWrapper(
    tf.nn.rnn_cell.MultiRNNCell([rnn_cell_with_dropout() for _ in range(num_layers)]),
    output_keep_prob=keep_prob)

output_series = []
state_series = []
initial_state = state = cell.zero_state(batch_size, tf.float32)
for current_input in input_series:
    output, state = cell(current_input, state)
    output_series.append(output)
    state_series.append(state)
first_state = state_series[0]
    
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, output_dim])
    b = tf.get_variable('b', [output_dim], initializer=tf.constant_initializer(0.0))

logits_series = [tf.matmul(output, W) + b for output in output_series]

# backward propagation

losses = [tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits) for logits, labels in zip(logits_series,label_series)]

# adjust for label imbalance and time
c = tf.constant([[7.143], [1.389], [7.143]])
# c = tf.constant([[7.143], [0.6945], [7.143]])
weight_series = []
for i in range(num_steps):
    weight_series.append((i+1)*0.1*tf.matmul(label_series[i], c))
losses = [tf.multiply(losses, weights) for losses, weights in zip(losses, weight_series)]

total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

# calculate accuracy

last_label = label_series[num_steps-1]
last_prediction = tf.nn.softmax(logits_series[num_steps-1])

label = tf.argmax(last_label, 1)
prediction = tf.argmax(last_prediction, 1)

equality = tf.equal(prediction, label)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

In [9]:
# run the training

epochs = 100

init_op = tf.global_variables_initializer()

with tf.Session() as sess:

    sess.run(init_op)
    
    for i in range(epochs):
        
        # training
        
        numpy_state = sess.run(initial_state)
    
        accumulated_acc = 0
        accumulated_loss = 0
        training_iterations = X_train.shape[0]
        for j in range(training_iterations):
            _, l, numpy_state, acc = sess.run([train_step, total_loss, first_state, accuracy], feed_dict={X:X_train[j:j+1], y:y_train_onehot[j:j+1], initial_state: numpy_state, keep_prob: global_dropout})
            accumulated_loss += l
            accumulated_acc += acc
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, accumulated_loss / training_iterations, accumulated_acc / training_iterations * 100))
        
        # validation
        
        numpy_state = sess.run(initial_state)
        
        accumulated_acc = 0
        accumulated_loss = 0
        labels = np.array([])
        predictions = np.array([])
        validation_iterations = X_valid.shape[0]
        for j in range(validation_iterations):
            l, numpy_state, acc, label_, prediction_ = sess.run([total_loss, first_state, accuracy, label, prediction], feed_dict={X:X_valid[j:j+1], y:y_valid_onehot[j:j+1], initial_state: numpy_state, keep_prob: 1.0})
            accumulated_loss += l
            accumulated_acc += acc
            labels = np.concatenate((labels, label_))
            predictions = np.concatenate((predictions, prediction_))
        print("Epoch: {}, loss: {:.3f}, validation accuracy: {:.2f}%".format(i, accumulated_loss / validation_iterations, accumulated_acc / validation_iterations * 100))
        conf = confusion_matrix(labels, predictions)
        print(conf)

Epoch: 0, loss: 1.698, training accuracy: 45.27%
Epoch: 0, loss: 1.435, validation accuracy: 55.92%
[[ 129   97   13]
 [ 560 1045  135]
 [  35  135   63]]
Epoch: 1, loss: 1.647, training accuracy: 47.51%
Epoch: 1, loss: 1.420, validation accuracy: 57.73%
[[ 121  102   16]
 [ 495 1078  167]
 [  29  126   78]]
Epoch: 2, loss: 1.635, training accuracy: 47.43%
Epoch: 2, loss: 1.421, validation accuracy: 63.92%
[[  97  126   16]
 [ 348 1250  142]
 [  24  142   67]]
Epoch: 3, loss: 1.632, training accuracy: 47.20%
Epoch: 3, loss: 1.437, validation accuracy: 63.34%
[[  92  129   18]
 [ 349 1242  149]
 [  27  139   67]]
Epoch: 4, loss: 1.627, training accuracy: 47.69%
Epoch: 4, loss: 1.439, validation accuracy: 64.29%
[[  78  136   25]
 [ 307 1267  166]
 [  23  133   77]]
Epoch: 5, loss: 1.623, training accuracy: 48.00%
Epoch: 5, loss: 1.413, validation accuracy: 66.64%
[[  71  146   22]
 [ 251 1323  166]
 [  24  129   80]]
Epoch: 6, loss: 1.621, training accuracy: 48.29%
Epoch: 6, loss: 1.402

Epoch: 52, loss: 1.691, validation accuracy: 66.27%
[[  59  148   32]
 [ 229 1337  174]
 [  24  139   70]]
Epoch: 53, loss: 1.387, training accuracy: 54.24%
Epoch: 53, loss: 1.691, validation accuracy: 66.05%
[[  52  149   38]
 [ 217 1335  188]
 [  19  140   74]]
Epoch: 54, loss: 1.383, training accuracy: 53.74%
Epoch: 54, loss: 1.696, validation accuracy: 65.10%
[[  57  144   38]
 [ 224 1313  203]
 [  21  142   70]]
Epoch: 55, loss: 1.384, training accuracy: 54.00%
Epoch: 55, loss: 1.713, validation accuracy: 66.59%
[[  61  146   32]
 [ 226 1341  173]
 [  16  146   71]]
Epoch: 56, loss: 1.376, training accuracy: 53.50%
Epoch: 56, loss: 1.696, validation accuracy: 66.64%
[[  67  146   26]
 [ 227 1342  171]
 [  25  143   65]]
Epoch: 57, loss: 1.370, training accuracy: 54.13%
Epoch: 57, loss: 1.722, validation accuracy: 66.09%
[[  63  146   30]
 [ 240 1330  170]
 [  23  141   69]]
Epoch: 58, loss: 1.361, training accuracy: 54.73%
Epoch: 58, loss: 1.755, validation accuracy: 66.27%
[[  63